In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
!pip install dataframe_image 
import dataframe_image as dfi

     |████████████████████████████████| 1.3MB 8.9MB/s 
     |████████████████████████████████| 296kB 49.7MB/s 
     |████████████████████████████████| 143kB 46.0MB/s 


In [ ]:
hospitals = pd.read_csv(r"/content/Hospitals.csv")
hospital_ratings = pd.read_csv(r"/content/Hospital General Information.csv", encoding="ISO-8859-1")
public_schools = pd.read_csv(r"/content/Public_Schools.csv")
county_time_series = pd.read_csv(r"/content/County_time_series.csv")
crosswalk = pd.read_csv(r"/content/CountyCrossWalk_Zillow.csv")
unemployment = pd.read_csv(r"/content/GeoFRED_Unemployment_Rate_by_County_Percent.csv")
crime_rate= pd.read_csv(r"/content/crime_data_w_population_and_crime_rate.csv")

In [ ]:
unemployment.head()

,Series ID,Region Name,Region Code,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,SCABBE1URN,"Abbeville County, SC",45001,3.8,6.6,7.7,9.1,8.2,7.7,8.8,8.1,8.1,14.2,13.6,12.5,10.6,9.5,7.7,6.9,5.5,4.6,4.2
1,LAACAD0URN,"Acadia Parish, LA",22001,5.8,6.0,6.3,6.8,5.8,6.1,3.6,3.8,4.2,6.5,7.3,6.7,6.0,6.0,5.8,6.9,7.4,5.9,5.4
2,VAACCO1URN,"Accomack County, VA",51001,3.0,3.4,4.2,4.4,4.7,4.7,4.3,4.1,5.0,6.5,7.9,8.4,7.9,7.2,6.7,5.5,5.0,4.9,4.1
3,IDADAC1URN,"Ada County, ID",16001,3.4,3.9,4.7,4.9,4.1,3.3,2.7,2.7,4.9,8.9,8.3,7.3,6.2,5.2,4.2,3.7,3.3,2.8,2.4
4,OKADAI1URN,"Adair County, OK",40001,3.5,4.1,4.8,7.8,6.2,4.8,4.4,4.7,4.9,8.0,10.5,9.2,9.1,9.8,8.0,6.1,5.9,5.0,4.4


In [ ]:
hospitals.head()

,X,Y,OBJECTID,ID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,TELEPHONE,TYPE,STATUS,POPULATION,COUNTY,COUNTYFIPS,COUNTRY,LATITUDE,LONGITUDE,NAICS_CODE,NAICS_DESC,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,WEBSITE,STATE_ID,ALT_NAME,ST_FIPS,OWNER,TTL_STAFF,BEDS,TRAUMA,HELIPAD
0,-94.945477,29.747620,8497,76777520,HOUSTON METHODIST SAN JACINTO HOSPITAL ALEXAND...,1700 JAMES BOWIE DRIVE,BAYTOWN,TX,77520,NOT AVAILABLE,(281) 420-8765,GENERAL ACUTE CARE,OPEN,182,HARRIS,48201,USA,29.747620,-94.945477,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.dshs.texas.gov/facilities/find-a-li...,2017-12-14T00:00:00.000Z,IMAGERY,2017-12-18T00:00:00.000Z,http://www.houstonmethodist.org/locations/san-...,NOT AVAILABLE,NOT AVAILABLE,48.0,NON-PROFIT,-999.0,182.0,NOT AVAILABLE,Y
1,-82.881843,40.027143,8498,129043230,"WOODS AT PARKSIDE,THE",349 OLDE RIDENOUR ROAD,COLUMBUS,OH,43230,NOT AVAILABLE,(614) 471-2552,SPECIAL,OPEN,50,FRANKLIN,39049,USA,40.027143,-82.881843,622210,"HOSPITALS, SUBSTANCE ABUSE",http://publicapps.odh.ohio.gov/EID/Default.aspx,2018-03-15T00:00:00.000Z,IMAGERY WITH OTHER,2018-04-26T00:00:00.000Z,http://www.thewoodsatparkside.com/,1815,NOT AVAILABLE,39.0,PROPRIETARY,-999.0,50.0,NOT AVAILABLE,NOT AVAILABLE
2,-84.168027,39.774242,8499,130045404,DAYTON CHILDREN'S HOSPITAL,ONE CHILDRENS PLAZA,DAYTON,OH,45404,NOT AVAILABLE,(937) 641-3300,CHILDREN,OPEN,155,MONTGOMERY,39113,USA,39.774242,-84.168027,622310,"CHILDREN'S HOSPITALS, SPECIALTY (EXCEPT PSYCHI...",http://publicapps.odh.ohio.gov/EID/Default.aspx,2018-03-15T00:00:00.000Z,IMAGERY WITH OTHER,2018-04-26T00:00:00.000Z,http://www.childrensdayton.org/cms/home/index....,1411,NOT AVAILABLE,39.0,NON-PROFIT,-999.0,155.0,PEDIATRIC LEVEL II,Y
3,-80.632972,41.005169,8500,128844512,VIBRA HOSPITAL OF MAHONING VALLEY,8049 SOUTH AVENUE,BOARDMAN,OH,44512,NOT AVAILABLE,(330) 726-5021,LONG TERM CARE,OPEN,45,MAHONING,39099,USA,41.005169,-80.632972,622310,"EXTENDED CARE HOSPITALS (EXCEPT MENTAL, SUBSTA...",http://publicapps.odh.ohio.gov/EID/Default.aspx,2018-03-15T00:00:00.000Z,IMAGERY WITH OTHER,2018-04-26T00:00:00.000Z,http://www.mahoningvalleyhospital.com/,1428,MAHONING VALLEY HOSPITAL BOARDMAN CAMPUS,39.0,PROPRIETARY,-999.0,45.0,NOT AVAILABLE,NOT AVAILABLE
4,-84.199398,39.747740,8501,129845417,HAVEN BEHAVIORAL SENIOR CARE OF DAYTON,"ONE ELIZABETH PLACE,E3 SUITE A",DAYTON,OH,45417,NOT AVAILABLE,(615) 393-8879,PSYCHIATRIC,OPEN,32,MONTGOMERY,39113,USA,39.747740,-84.199398,622210,"HOSPITALS, PSYCHIATRIC (EXCEPT CONVALESCENT)",http://publicapps.odh.ohio.gov/EID/Default.aspx,2018-03-15T00:00:00.000Z,IMAGERY WITH OTHER,2018-04-26T00:00:00.000Z,https://dayton.havenbehavioral.com/,1506,NOT AVAILABLE,39.0,PROPRIETARY,-999.0,32.0,NOT AVAILABLE,NOT AVAILABLE


In [ ]:
hospital_ratings.head(4)

,Provider ID,Hospital Name,Address,City,State,ZIP Code,County Name,Phone Number,Hospital Type,Hospital Ownership,Emergency Services,Meets criteria for meaningful use of EHRs,Hospital overall rating,Hospital overall rating footnote,Mortality national comparison,Mortality national comparison footnote,Safety of care national comparison,Safety of care national comparison footnote,Readmission national comparison,Readmission national comparison footnote,Patient experience national comparison,Patient experience national comparison footnote,Effectiveness of care national comparison,Effectiveness of care national comparison footnote,Timeliness of care national comparison,Timeliness of care national comparison footnote,Efficient use of medical imaging national comparison,Efficient use of medical imaging national comparison footnote
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,3347938701,Acute Care Hospitals,Government - Hospital District or Authority,Yes,Y,3,NaN,Same as the national average,NaN,Above the national average,NaN,Same as the national average,NaN,Below the national average,NaN,Same as the national average,NaN,Same as the national average,NaN,Same as the national average,NaN
1,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,2565938310,Acute Care Hospitals,Government - Hospital District or Authority,Yes,Y,3,NaN,Below the national average,NaN,Same as the national average,NaN,Above the national average,NaN,Same as the national average,NaN,Same as the national average,NaN,Above the national average,NaN,Below the national average,NaN
2,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,LAUDERDALE,2567688400,Acute Care Hospitals,Government - Hospital District or Authority,Yes,Y,2,NaN,Below the national average,NaN,Same as the national average,NaN,Same as the national average,NaN,Below the national average,NaN,Same as the national average,NaN,Above the national average,NaN,Same as the national average,NaN
3,10007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,3344933541,Acute Care Hospitals,Voluntary non-profit - Private,Yes,Y,2,NaN,Same as the national average,NaN,Not Available,Results are not available for this reporting p...,Below the national average,NaN,Same as the national average,NaN,Below the national average,NaN,Above the national average,NaN,Not Available,Results are not available for this reporting p...


In [ ]:
hospital_ratings = hospital_ratings[['Hospital Name', 'Hospital overall rating']]
hospital_ratings.dropna()
hospital_ratings.head()

,Hospital Name,Hospital overall rating
0,SOUTHEAST ALABAMA MEDICAL CENTER,3
1,MARSHALL MEDICAL CENTER SOUTH,3
2,ELIZA COFFEE MEMORIAL HOSPITAL,2
3,MIZELL MEMORIAL HOSPITAL,2
4,CRENSHAW COMMUNITY HOSPITAL,3


In [ ]:
hospitals_with_ratings = pd.merge(hospitals, hospital_ratings, how='left', left_on='NAME', right_on='Hospital Name')

hospitals_with_ratings = hospitals_with_ratings.loc[hospitals_with_ratings['COUNTYFIPS'] != 'NOT AVAILABLE']

hospitals_with_ratings = hospitals_with_ratings.loc[~pd.isna(hospitals_with_ratings)['Hospital overall rating']]
hospitals_with_ratings = hospitals_with_ratings.loc[hospitals_with_ratings['Hospital overall rating'] != 'Not Available']
hospitals_with_ratings['Hospital overall rating'] = hospitals_with_ratings['Hospital overall rating'].astype("int64")

hospitals_with_ratings.rename(index=str, columns={"COUNTYFIPS": "FIPS", "Hospital overall rating": "AverageHospitalRating"}, inplace=True)

hospitals_with_ratings.head(2)

,X,Y,OBJECTID,ID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,TELEPHONE,TYPE,STATUS,POPULATION,COUNTY,FIPS,COUNTRY,LATITUDE,LONGITUDE,NAICS_CODE,NAICS_DESC,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,WEBSITE,STATE_ID,ALT_NAME,ST_FIPS,OWNER,TTL_STAFF,BEDS,TRAUMA,HELIPAD,Hospital Name,AverageHospitalRating
7,-101.294085,48.232226,8504,5258701,TRINITY HOSPITALS,1 W BURDICK EXPY,MINOT,ND,58701,NOT AVAILABLE,(701) 857-5000,GENERAL ACUTE CARE,OPEN,251,WARD,38101,USA,48.232226,-101.294085,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.ndhealth.gov/hf/North_Dakota_Hospit...,2017-09-11T00:00:00.000Z,IMAGERY,2017-10-02T00:00:00.000Z,http://www.trinityhealth.org,5055A,NOT AVAILABLE,38.0,NON-PROFIT,-999.0,251.0,LEVEL II,Y,TRINITY HOSPITALS,1
41,-78.855473,38.404787,8528,147322801,SENTARA RMH MEDICAL CENTER,2010 HEALTH CAMPUS DRIVE,HARRISONBURG,VA,22801,NOT AVAILABLE,(540) 689-1000,GENERAL ACUTE CARE,OPEN,238,ROCKINGHAM,51165,USA,38.404787,-78.855473,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.vhi.org,2017-11-13T00:00:00.000Z,IMAGERY WITH OTHER,2018-01-26T00:00:00.000Z,www.rmhonline.com,NOT AVAILABLE,ROCKINGHAM MEMORIAL HOSPITAL,51.0,NON-PROFIT,-999.0,238.0,NOT AVAILABLE,NOT AVAILABLE,SENTARA RMH MEDICAL CENTER,3


In [ ]:
unemployment_per_county = unemployment.loc[:,("Region Code", "2018")]
unemployment_per_county.rename(index=str, columns={"Region Code": "FIPS", "2018": "UnemploymentRate"}, inplace=True)
unemployment_per_county["FIPS"] = unemployment_per_county["FIPS"].astype("int64")

unemployment_per_county.head()

,FIPS,UnemploymentRate
0,45001,4.2
1,22001,5.4
2,51001,4.1
3,16001,2.4
4,40001,4.4


In [ ]:
average_hospital_rating_per_county = hospitals_with_ratings[['FIPS', 'AverageHospitalRating']]

average_hospital_rating_per_county = average_hospital_rating_per_county.dropna()

average_hospital_rating_per_county = average_hospital_rating_per_county.groupby(['FIPS']).mean()
average_hospital_rating_per_county.reset_index(level=0, inplace=True)
average_hospital_rating_per_county["FIPS"] = average_hospital_rating_per_county["FIPS"].astype("int64")

average_hospital_rating_per_county.head()

,FIPS,AverageHospitalRating
0,10005,3.5
1,1001,4.0
2,1003,3.0
3,1005,3.0
4,1011,3.0


In [ ]:
hospitals_per_county = hospitals['COUNTYFIPS'].value_counts().to_frame()
hospitals_per_county.reset_index(level=0, inplace=True)
hospitals_per_county.rename(index=str, columns={"index": "FIPS", "COUNTYFIPS": "NumberOfHospitals"}, inplace=True)
hospitals_per_county.dropna(inplace=True)
hospitals_per_county= hospitals_per_county[hospitals_per_county["FIPS"] != "NOT AVAILABLE"]
hospitals_per_county["FIPS"] = hospitals_per_county["FIPS"].astype("int64")

hospitals_per_county.head()

,FIPS,NumberOfHospitals
0,6037,131
1,48201,93
2,4013,73
3,17031,73
4,48113,58


In [ ]:
hospitals_per_county = hospitals_per_county.merge(average_hospital_rating_per_county, on="FIPS", how="left")
hospitals_per_county.head()

,FIPS,NumberOfHospitals,AverageHospitalRating
0,6037,131,2.775000
1,48201,93,3.357143
2,4013,73,3.047619
3,17031,73,2.695652
4,48113,58,3.222222


In [ ]:
county_time_series.head()

,Date,RegionName,DaysOnZillow_AllHomes,InventorySeasonallyAdjusted_AllHomes,InventoryRaw_AllHomes,MedianListingPricePerSqft_1Bedroom,MedianListingPricePerSqft_2Bedroom,MedianListingPricePerSqft_3Bedroom,MedianListingPricePerSqft_4Bedroom,MedianListingPricePerSqft_5BedroomOrMore,MedianListingPricePerSqft_AllHomes,MedianListingPricePerSqft_CondoCoop,MedianListingPricePerSqft_DuplexTriplex,MedianListingPricePerSqft_SingleFamilyResidence,MedianListingPrice_1Bedroom,MedianListingPrice_2Bedroom,MedianListingPrice_3Bedroom,MedianListingPrice_4Bedroom,MedianListingPrice_5BedroomOrMore,MedianListingPrice_AllHomes,MedianListingPrice_CondoCoop,MedianListingPrice_DuplexTriplex,MedianListingPrice_SingleFamilyResidence,MedianPctOfPriceReduction_AllHomes,MedianPctOfPriceReduction_CondoCoop,MedianPctOfPriceReduction_SingleFamilyResidence,MedianPriceCutDollar_AllHomes,MedianPriceCutDollar_CondoCoop,MedianPriceCutDollar_SingleFamilyResidence,MedianRentalPricePerSqft_1Bedroom,MedianRentalPricePerSqft_2Bedroom,MedianRentalPricePerSqft_3Bedroom,MedianRentalPricePerSqft_4Bedroom,MedianRentalPricePerSqft_5BedroomOrMore,MedianRentalPricePerSqft_AllHomes,MedianRentalPricePerSqft_CondoCoop,MedianRentalPricePerSqft_DuplexTriplex,MedianRentalPricePerSqft_MultiFamilyResidence5PlusUnits,MedianRentalPricePerSqft_SingleFamilyResidence,MedianRentalPricePerSqft_Studio,...,MedianRentalPrice_3Bedroom,MedianRentalPrice_4Bedroom,MedianRentalPrice_5BedroomOrMore,MedianRentalPrice_AllHomes,MedianRentalPrice_CondoCoop,MedianRentalPrice_DuplexTriplex,MedianRentalPrice_MultiFamilyResidence5PlusUnits,MedianRentalPrice_SingleFamilyResidence,MedianRentalPrice_Studio,ZHVIPerSqft_AllHomes,PctOfHomesDecreasingInValues_AllHomes,PctOfHomesIncreasingInValues_AllHomes,PctOfHomesSellingForGain_AllHomes,PctOfHomesSellingForLoss_AllHomes,PctOfListingsWithPriceReductionsSeasAdj_AllHomes,PctOfListingsWithPriceReductionsSeasAdj_CondoCoop,PctOfListingsWithPriceReductionsSeasAdj_SingleFamilyResidence,PctOfListingsWithPriceReductions_AllHomes,PctOfListingsWithPriceReductions_CondoCoop,PctOfListingsWithPriceReductions_SingleFamilyResidence,PriceToRentRatio_AllHomes,Sale_Counts,Sale_Counts_Seas_Adj,Sale_Prices,ZHVI_1bedroom,ZHVI_2bedroom,ZHVI_3bedroom,ZHVI_4bedroom,ZHVI_5BedroomOrMore,ZHVI_AllHomes,ZHVI_BottomTier,ZHVI_CondoCoop,ZHVI_MiddleTier,ZHVI_SingleFamilyResidence,ZHVI_TopTier,ZRI_AllHomes,ZRI_AllHomesPlusMultifamily,ZriPerSqft_AllHomes,Zri_MultiFamilyResidenceRental,Zri_SingleFamilyResidenceRental
0,1996-04-30,10001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69200.0,88700.0,125900.0,135600.0,85700.0,65600.0,NaN,85700.0,86000.0,129700.0,NaN,NaN,NaN,NaN,NaN
1,1996-04-30,10003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96700.0,93000.0,106300.0,167400.0,NaN,116800.0,81900.0,NaN,116800.0,116900.0,181400.0,NaN,NaN,NaN,NaN,NaN
2,1996-04-30,1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97700.0,103100.0,162400.0,NaN,110600.0,70600.0,NaN,110600.0,106800.0,192300.0,NaN,NaN,NaN,NaN,NaN
3,1996-04-30,1009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70000.0,NaN,NaN,70000.0,70000.0,123500.0,NaN,NaN,NaN,NaN,NaN
4,1996-04-30,1017,NaN,NaN,NaN,NaN,N

In [ ]:
crosswalk.head()

,CountyName,StateName,StateFIPS,CountyFIPS,MetroName_Zillow,CBSAName,CountyRegionID_Zillow,MetroRegionID_Zillow,FIPS,CBSACode
0,Pike,Pennsylvania,42,103,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",280,394913.0,42103,35620.0
1,Bronx,New York,36,5,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",401,394913.0,36005,35620.0
2,Essex,New Jersey,34,13,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",504,394913.0,34013,35620.0
3,Kings,New York,36,47,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",581,394913.0,36047,35620.0
4,Ocean,New Jersey,34,29,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",659,394913.0,34029,35620.0


In [ ]:
crosswalk = crosswalk[['FIPS', 'CountyName', 'StateName']]
crosswalk["FIPS"] = crosswalk["FIPS"].astype("int64")
crosswalk.head()

,FIPS,CountyName,StateName
0,42103,Pike,Pennsylvania
1,36005,Bronx,New York
2,34013,Essex,New Jersey
3,36047,Kings,New York
4,34029,Ocean,New Jersey


In [ ]:
house_prices = county_time_series.groupby("RegionName").mean()
team_est = house_prices[["ZHVI_AllHomes","ZHVIPerSqft_AllHomes","MedianListingPricePerSqft_AllHomes","MedianRentalPrice_AllHomes"]]
team_est.reset_index(level=0, inplace=True)
team_est.rename(index=str, columns={"RegionName": "FIPS", "ZHVI_AllHomes": "AverageHousePrice"}, inplace=True)
team_est["FIPS"] = team_est["FIPS"].astype("int64")
team_est.dropna(inplace=True)

team_est.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,FIPS,AverageHousePrice,ZHVIPerSqft_AllHomes,MedianListingPricePerSqft_AllHomes,MedianRentalPrice_AllHomes
0,1001,114483.665339,72.561753,98.335546,1255.208333
1,1003,164861.685824,90.390805,131.896040,1165.939759
28,1069,114940.996169,72.938697,90.650996,975.625000
30,1073,99759.770115,68.245211,91.358103,855.860294
33,1081,148163.218391,74.789272,95.511291,1143.887097


In [ ]:
public_schools.head()

,X,Y,OBJECTID,NCESID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,TELEPHONE,TYPE,STATUS,POPULATION,COUNTY,COUNTYFIPS,COUNTRY,LATITUDE,LONGITUDE,NAICS_CODE,NAICS_DESC,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,WEBSITE,LEVEL_,ENROLLMENT,ST_GRADE,END_GRADE,DISTRICTID,FT_TEACHER,SHELTER_ID
0,-81.050895,29.022271,2002,120192008041,SAMSULA ACADEMY,248 N SAMSULA DR,NEW SMYRNA,FL,32168,8762,(386) 423-6650,1,1,235,VOLUSIA,12127,USA,29.022271,-81.050895,611110,ELEMENTARY AND SECONDARY SCHOOLS,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,2014-02-04T00:00:00.000Z,IMAGERY,2014-05-20T00:00:00.000Z,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,222,KG,05,1201920,13,NOT AVAILABLE
1,-92.507288,31.180659,2003,220129002344,CAROLINE DORMON JUNIOR HIGH SCHOOL,8906 HWY 165 SOUTH,WOODWORTH,LA,71485,NOT AVAILABLE,(318) 473-4066,1,1,333,RAPIDES,22079,USA,31.180659,-92.507288,611110,ELEMENTARY AND SECONDARY SCHOOLS,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,2015-05-08T00:00:00.000Z,IMAGERY/OTHER,2015-06-19T00:00:00.000Z,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,312,PK,08,2201290,21,NOT AVAILABLE
2,-69.971880,43.908147,2004,230378023129,HARRIET BEECHER STOWE ELEMENTARY,44 MCKEEN STREET,BRUNSWICK,ME,4011,NOT AVAILABLE,(207) 319-1960,1,1,791,CUMBERLAND,23005,USA,43.908147,-69.971880,611110,ELEMENTARY AND SECONDARY SCHOOLS,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,2014-02-04T00:00:00.000Z,IMAGERY,2014-05-07T00:00:00.000Z,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,739,02,05,2303780,52,NOT AVAILABLE
3,-89.542799,32.728496,2005,280252001118,LEAKE CENTRAL ELEMENTARY SCHOOL,603 HWY. 16 WEST,CARTHAGE,MS,39051,NOT AVAILABLE,(601) 267-9148,1,1,1222,LEAKE,28079,USA,32.728496,-89.542799,611110,ELEMENTARY AND SECONDARY SCHOOLS,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,2009-10-13T00:00:00.000Z,IMAGERY/OTHER,2010-07-06T00:00:00.000Z,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,1159,PK,05,2802520,63,NOT AVAILABLE
4,-94.361775,39.364359,2006,291645000891,KEARNEY ELEM.,902 S JEFFERSON,KEARNEY,MO,64060,8518,(816) 628-4113,1,1,296,CLAY,29047,USA,39.364359,-94.361775,611110,ELEMENTARY AND SECONDARY SCHOOLS,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,2009-10-13T00:00:00.000Z,IMAGERY/OTHER,2016-07-18T00:00:00.000Z,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,274,KG,05,2916450,22,NOT AVAILABLE


In [ ]:
public_schools_per_county = public_schools['COUNTYFIPS'].value_counts().to_frame()
public_schools_per_county.reset_index(level=0, inplace=True)
public_schools_per_county.rename(index=str, columns={"index": "FIPS", "COUNTYFIPS": "NumberOfSchools"}, inplace=True)
public_schools_per_county["FIPS"] = public_schools_per_county["FIPS"].astype("int64")


public_schools_per_county.head()

,FIPS,NumberOfSchools
0,6037,2277
1,17031,1326
2,4013,1246
3,48201,1125
4,6073,763


In [ ]:
team_est = team_est.merge(crosswalk, on="FIPS")

team_est.head()

,FIPS,AverageHousePrice,ZHVIPerSqft_AllHomes,MedianListingPricePerSqft_AllHomes,MedianRentalPrice_AllHomes,CountyName,StateName,StateFIPS,CountyFIPS,MetroName_Zillow,CBSAName,CountyRegionID_Zillow,MetroRegionID_Zillow,CBSACode
0,1001,114483.665339,72.561753,98.335546,1255.208333,Autauga,Alabama,1,1,"Montgomery, AL","Montgomery, AL",1524,394875.0,33860.0
1,1003,164861.685824,90.390805,131.896040,1165.939759,Baldwin,Alabama,1,3,"Daphne, AL","Daphne-Fairhope-Foley, AL",1525,394519.0,19300.0
2,1069,114940.996169,72.938697,90.650996,975.625000,Houston,Alabama,1,69,"Dothan, AL","Dothan, AL",1781,394537.0,20020.0
3,1073,99759.770115,68.245211,91.358103,855.860294,Jefferson,Alabama,1,73,"Birmingham, AL","Birmingham-Hoover, AL",2732,394388.0,13820.0
4,1081,148163.218391,74.789272,95.511291,1143.887097,Lee,Alabama,1,81,"Auburn, AL","Auburn-Opelika, AL",78,394351.0,12220.0


In [ ]:
team_est = team_est.merge(unemployment_per_county, on="FIPS")
team_est.head()

,FIPS,AverageHousePrice,ZHVIPerSqft_AllHomes,MedianListingPricePerSqft_AllHomes,MedianRentalPrice_AllHomes,CountyName,StateName,StateFIPS,CountyFIPS,MetroName_Zillow,CBSAName,CountyRegionID_Zillow,MetroRegionID_Zillow,CBSACode,UnemploymentRate
0,1001,114483.665339,72.561753,98.335546,1255.208333,Autauga,Alabama,1,1,"Montgomery, AL","Montgomery, AL",1524,394875.0,33860.0,3.6
1,1003,164861.685824,90.390805,131.896040,1165.939759,Baldwin,Alabama,1,3,"Daphne, AL","Daphne-Fairhope-Foley, AL",1525,394519.0,19300.0,3.6
2,1069,114940.996169,72.938697,90.650996,975.625000,Houston,Alabama,1,69,"Dothan, AL","Dothan, AL",1781,394537.0,20020.0,4.1
3,1073,99759.770115,68.245211,91.358103,855.860294,Jefferson,Alabama,1,73,"Birmingham, AL","Birmingham-Hoover, AL",2732,394388.0,13820.0,3.7
4,1081,148163.218391,74.789272,95.511291,1143.887097,Lee,Alabama,1,81,"Auburn, AL","Auburn-Opelika, AL",78,394351.0,12220.0,3.6


In [ ]:
team_est = team_est.merge(public_schools_per_county, on="FIPS")
team_est.head()

,FIPS,AverageHousePrice,ZHVIPerSqft_AllHomes,MedianListingPricePerSqft_AllHomes,MedianRentalPrice_AllHomes,CountyName,StateName,StateFIPS,CountyFIPS,MetroName_Zillow,CBSAName,CountyRegionID_Zillow,MetroRegionID_Zillow,CBSACode,UnemploymentRate,NumberOfSchools
0,1001,114483.665339,72.561753,98.335546,1255.208333,Autauga,Alabama,1,1,"Montgomery, AL","Montgomery, AL",1524,394875.0,33860.0,3.6,15
1,1003,164861.685824,90.390805,131.896040,1165.939759,Baldwin,Alabama,1,3,"Daphne, AL","Daphne-Fairhope-Foley, AL",1525,394519.0,19300.0,3.6,47
2,1069,114940.996169,72.938697,90.650996,975.625000,Houston,Alabama,1,69,"Dothan, AL","Dothan, AL",1781,394537.0,20020.0,4.1,31
3,1073,99759.770115,68.245211,91.358103,855.860294,Jefferson,Alabama,1,73,"Birmingham, AL","Birmingham-Hoover, AL",2732,394388.0,13820.0,3.7,226
4,1081,148163.218391,74.789272,95.511291,1143.887097,Lee,Alabama,1,81,"Auburn, AL","Auburn-Opelika, AL",78,394351.0,12220.0,3.6,37


In [ ]:
team_est = team_est.merge(hospitals_per_county, on="FIPS")
team_est.head()

,FIPS,AverageHousePrice,ZHVIPerSqft_AllHomes,MedianListingPricePerSqft_AllHomes,MedianRentalPrice_AllHomes,CountyName,StateName,StateFIPS,CountyFIPS,MetroName_Zillow,CBSAName,CountyRegionID_Zillow,MetroRegionID_Zillow,CBSACode,UnemploymentRate,NumberOfSchools,NumberOfHospitals,AverageHospitalRating
0,1001,114483.665339,72.561753,98.335546,1255.208333,Autauga,Alabama,1,1,"Montgomery, AL","Montgomery, AL",1524,394875.0,33860.0,3.6,15,1,4.00
1,1003,164861.685824,90.390805,131.896040,1165.939759,Baldwin,Alabama,1,3,"Daphne, AL","Daphne-Fairhope-Foley, AL",1525,394519.0,19300.0,3.6,47,4,3.00
2,1069,114940.996169,72.938697,90.650996,975.625000,Houston,Alabama,1,69,"Dothan, AL","Dothan, AL",1781,394537.0,20020.0,4.1,31,5,3.50
3,1073,99759.770115,68.245211,91.358103,855.860294,Jefferson,Alabama,1,73,"Birmingham, AL","Birmingham-Hoover, AL",2732,394388.0,13820.0,3.7,226,14,2.75
4,1081,148163.218391,74.789272,95.511291,1143.887097,Lee,Alabama,1,81,"Auburn, AL","Auburn-Opelika, AL",78,394351.0,12220.0,3.6,37,1,4.00


In [ ]:
team_est = team_est[['FIPS', 'CountyName', 'StateName', 'NumberOfSchools', 'NumberOfHospitals', 'AverageHospitalRating', 'UnemploymentRate',"ZHVIPerSqft_AllHomes", 'AverageHousePrice',"MedianRentalPrice_AllHomes"]]
team_est.head()

,FIPS,CountyName,StateName,NumberOfSchools,NumberOfHospitals,AverageHospitalRating,UnemploymentRate,ZHVIPerSqft_AllHomes,AverageHousePrice,MedianRentalPrice_AllHomes
0,1001,Autauga,Alabama,15,1,4.00,3.6,72.561753,114483.665339,1255.208333
1,1003,Baldwin,Alabama,47,4,3.00,3.6,90.390805,164861.685824,1165.939759
2,1069,Houston,Alabama,31,5,3.50,4.1,72.938697,114940.996169,975.625000
3,1073,Jefferson,Alabama,226,14,2.75,3.7,68.245211,99759.770115,855.860294
4,1081,Lee,Alabama,37,1,4.00,3.6,74.789272,148163.218391,1143.887097


In [ ]:
crime_rate.head()

,county_name,crime_rate_per_100000,index,EDITION,PART,IDNO,CPOPARST,CPOPCRIM,AG_ARRST,AG_OFF,COVIND,INDEX,MODINDX,MURDER,RAPE,ROBBERY,AGASSLT,BURGLRY,LARCENY,MVTHEFT,ARSON,population,FIPS_ST,FIPS_CTY
0,"St. Louis city, MO",1791.995377,1,1,4,1612,318667,318667,15,15,100.0,5706,22329,119,200,1778,3609,4995,13791,3543,464,318416,29,510
1,"Crittenden County, AR",1754.914968,2,1,4,130,50717,50717,4,4,100.0,873,3424,8,38,165,662,1482,1753,189,28,49746,5,35
2,"Alexander County, IL",1664.700485,3,1,4,604,8040,8040,2,2,100.0,127,278,1,2,5,119,82,184,12,2,7629,17,3
3,"Kenedy County, TX",1456.310680,4,1,4,2681,444,444,1,1,100.0,6,13,0,3,1,2,5,4,4,0,412,48,261
4,"De Soto Parish, LA",1447.402430,5,1,4,1137,26971,26971,3,3,100.0,392,703,3,4,17,368,149,494,60,0,27083,22,31


In [ ]:
crime_rate= crime_rate[["county_name","crime_rate_per_100000","FIPS_ST","FIPS_CTY"]]
crime_rate

,county_name,crime_rate_per_100000,FIPS_ST,FIPS_CTY
0,"St. Louis city, MO",1791.995377,29,510
1,"Crittenden County, AR",1754.914968,5,35
2,"Alexander County, IL",1664.700485,17,3
3,"Kenedy County, TX",1456.310680,48,261
4,"De Soto Parish, LA",1447.402430,22,31
...,...,...,...,...
3131,"Ohio County, IN",0.000000,18,115
3132,"Newton County, MS",0.000000,28,101
3133,"Jerauld County, SD",0.000000,46,73
3134,"Cimarron County, OK",0.000000,40,25


In [ ]:
team_est.to_csv(path_or_buf="\content")